In [1]:
# extract files from vectorstore_index.tgz
import tarfile
with tarfile.open("vectorstore_index.tgz", "r:gz") as tar:
    tar.extractall()



C:\Users\yanal\AppData\Local\Temp\ipykernel_3840\2704633124.py:4: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall()


In [5]:
from langchain_huggingface import HuggingFaceEmbeddings

# For E5 models, you need to add instruction prefixes
def get_e5_embeddings():
    model_name = "intfloat/multilingual-e5-large"  # or "multilingual-e5-base" for faster
    model_kwargs = {'device': 'cpu'}  # Use 'cpu' if no GPU
    encode_kwargs = {'normalize_embeddings': True}  # Important for cosine similarity

    embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    return embeddings

embeddings = get_e5_embeddings()

c:\Users\yanal\OneDrive\Desktop\Project in ML repo\Project_in_ML\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from langchain_chroma import Chroma

# load vector store from vectorstore_index.tgz
persist_dir = "chroma_db"  # folder extracted from .tgz

vectorstore = Chroma(
    persist_directory=persist_dir,
    embedding_function=embeddings
)

In [1]:
# extract files from vectorstore_index.tgz
import tarfile
with tarfile.open("db_schemas.tgz", "r:gz") as tar:
    tar.extractall()



C:\Users\yanal\AppData\Local\Temp\ipykernel_25852\1944152321.py:4: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall()


## Query type classification

In [1]:
sql_queries = [
    "מה הציון הממוצע בקורס מבני נתונים?",
    "מה הציונים בשנים קודמות של מבני נתונים?",
    "איך השתנה הציון הממוצע בקורס לאורך השנים?",
    "איזה קורסים עם הציון הממוצע הכי נמוך?",
    "דירוג קורסים לפי ממוצע ציונים",
    "מה הציון הממוצע של דני קרן בחדוא 1 בשנה שעברה?",
    "האם יש שיפור בציונים בקורס אלגוריתמים עם משה לוי?",
    "עשרת הממוצעים הכי גבוהים בקורס למידה עמוקה עם ריטה ב 3 שנים אחרונות",
    "מה הקורסים שצריך לפני מבני נתונים?",
    "מהם הקדמים של אלגוריתמים?",
    "איזה קורסים חובה לפני קורס מסוים?",
    "מה צריך ללמוד לפני קורס מערכות הפעלה?",
    "מה הקדמים של מבוא למדעי המחשב?",
    "האם לינארית 1 היא קדם לאנליזה 2?",
    "What is the average grade in Databases?",
    "SELECT the top 5 courses with the highest average grade",
    "How many students passed the AI course?",
    "List all students who failed Operating Systems",
    "מה הציון הממוצע בקורס מבני נתונים?",
    "מה הקורס עם מספר המתקדמים הגבוה ביותר?",
    "List courses that require Calculus as a prerequisite",
    "How many courses require Algebra 2 as prerequisite?",
    "מהם הקורסים המצריכים לאלגוריתמים?",
    "What is the maximum average in Computer Networks?",
    "Show students with grade above 90 in Databases",
    "מה הם הממוצעים המינימליים בקורס מערכות הפעלה?",
    "List all courses that have no prerequisites",
    "What are the prerequisites for reinforcement learning",
]

semantic_queries = [
    "חוות דעת על קורס מבני נתונים",
    "מה הסטודנטים אומרים על קורס למידה עמוקה?",
    "Why do students struggle with Computer Networks?",
    "What do students think about Artificial Intelligence?",
    "מרצים מומלצים",
    "חוות דעת על דני קרן",
    "האם קורס אלגוריתמים קשה?",
    "איך אני יכול לשפר את הציונים שלי בקורסים?",
    "What do students think about Databases course?",
    "Which professor is recommended for AI course?",
    "How difficult is Operating Systems?",
    "מה דעתכם על הקורס מבוא למבני נתונים?",
    "Which courses in the prerequisites table are most interesting?",
    "מה חוות הדעת של הסטודנטים על הקורס למידת מכונה?",
    "Is Computer Networks course more practical than Database Systems?",
    "Which courses require more preparation according to students?",
    "מהם הקורסים הכי מומלצים ללימודי הסמינר?",
    "Explain why the AI course is challenging",
    "How do students rate the Algebra 1 course?",
    "מהן ההמלצות לגבי פרויקט גמר בקורס למידה עמוקה?",
    "Which courses are easier to pass",
    "מה כדאי לדעת לפני שמתחילים את קורס תורת המשחקים?",
    "קורסים עם תרגילי בית קשים",
    "קורסים עם תרגילי בית שצריכים השקעה ומאמץ",
    "חוות דעת על קורס מבוא לחומרה",
    "מרצים טובים ומומלצים",
    "קורסים שמסתכלים עליהם בעבודה"
]

print("Query lists defined successfully.")

Query lists defined successfully.


In [2]:
from langchain_core.documents import Document

classification_documents = []

# SQL 'q' (grades) queries
for query in sql_queries:
    classification_documents.append(
        Document(
            page_content=f"passage: {query}",
            metadata={
                "route": "sql"
            }
        )
    )


# Semantic queries
for query in semantic_queries:
    classification_documents.append(
        Document(
            page_content=f"passage: {query}",
            metadata={
                "route": "semantic"
            }
        )
    )

print(f"Created {len(classification_documents)} classification documents.")
# Optionally, print a few to verify
# for i, doc in enumerate(classification_documents[:5]):
#     print(f"\nDocument {i+1}:\n  Content: {doc.page_content}\n  Metadata: {doc.metadata}")

Created 55 classification documents.


In [3]:
from sentence_transformers import SentenceTransformer
from langchain_core.embeddings import Embeddings
from langchain_chroma import Chroma  # or use FAISS, Pinecone, etc.

class E5Embeddings(Embeddings):
    def __init__(self, model_name="intfloat/multilingual-e5-large"):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts):
        texts = [f"passage: {t}" for t in texts]
        return self.model.encode(
            texts,
            normalize_embeddings=True
        ).tolist()

    def embed_query(self, text):
        return self.model.encode(
            [f"query: {text}"],
            normalize_embeddings=True
        )[0].tolist()

def create_vector_store(docs_split, embeddings, persist_directory="./chroma_db"):
    vectorstore = Chroma.from_documents(
        documents=docs_split,
        embedding=embeddings,
        persist_directory=persist_directory
    )
    return vectorstore

embeddings = E5Embeddings()
classification_vectorstore = create_vector_store(
    classification_documents,
    embeddings,
    persist_directory="./classification_db"
)

print("Classification vector store created at ./classification_db")

!tar czvf classification_db.tgz classification_db

c:\Users\yanal\OneDrive\Desktop\Project in ML repo\Project_in_ML\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Classification vector store created at ./classification_db


a classification_db
a classification_db/4941dc91-19b2-400d-bdcf-d280e8b7c63d
a classification_db/chroma.sqlite3
a classification_db/4941dc91-19b2-400d-bdcf-d280e8b7c63d/data_level0.bin
a classification_db/4941dc91-19b2-400d-bdcf-d280e8b7c63d/header.bin
a classification_db/4941dc91-19b2-400d-bdcf-d280e8b7c63d/length.bin
a classification_db/4941dc91-19b2-400d-bdcf-d280e8b7c63d/link_lists.bin


In [ ]:
def classify_query(query: str, schema_vectorstore):
    results = schema_vectorstore.similarity_search_with_score(query, k=1)

    if not results:
        return {"route": "semantic", "score": None, "table": None}

    doc, _ = results[0]

    # return the same type as the closest query
    return doc.metadata["route"]

query_sql = "מה הציון הממוצע בקורס אלגוריתמים?"
query_semantic = "מרצים טובים ומומלצים"

print(f"Classifying query: '{query_sql}'")
result_sql = classify_query(query_sql, classification_vectorstore, threshold=0.4)
print(f"Result: {result_sql}\n")

print(f"Classifying query: '{query_semantic}'")
result_semantic = classify_query(query_semantic, classification_vectorstore, threshold=0.4)
print(f"Result: {result_semantic}\n")

Classifying query: 'מה הציון הממוצע בקורס אלגוריתמים?'
Result: sql

Classifying query: 'מרצים טובים ומומלצים'
Result: semantic



In [5]:
queries = [
    "חוות דעת על קורס שפות סף",
    "עשרת הציונים הכי גבוהים במבני נתונים",
    "מה הן הקדמים של קורס הסתברות",
    "מה הסטודנטים אומרים על המרצה אורן וימן"
]

# test
for query in  queries:
  print(f"Classifying query: '{query}'")
  result = classify_query(query, classification_vectorstore, threshold=0.4)
  print(f"Result: {result}\n")

Classifying query: 'חוות דעת על קורס שפות סף'
Result: semantic

Classifying query: 'עשרת הציונים הכי גבוהים במבני נתונים'
Result: sql

Classifying query: 'מה הן הקדמים של קורס הסתברות'
Result: sql

Classifying query: 'מה הסטודנטים אומרים על המרצה אורן וימן'
Result: semantic

